# Documentation

**In a nutshell**

This notebook documents an approach to derive Global Warming Potential (GWP) Emission Factors (EFs) using IPCC AR5 for all sectors and regions from the publicly available database Exiobase 3.8.2 *product x product* for the year 2019 **[1]**
This document represents the calculation script to derive EFs from Exiobase raw data and some subsequent data cleaning (i.e. removing what I believe are artifacts); I've tried to integrate meaningful comments.

**Approach:**
1. Set up: setting year and type (pxp or ixi) of raw data zip, parsing Exiobase raw data, and calculating the system and extension results (using calc_all() from pymrio).

1. Calculating the GWP Emission Factors for all sectors and regions - in a loop iterating over all sectors and regions:
    1. Creating a demand vector for sector i and region j - a vector with all zeros except for sector i and region j which equals to 1
    1. Creating a scaling vector through multiplication of the Leontief matrix with the demand vector
    1. Calculating a GWP Emission Factor through matrix-multiplication of a pseudo / depreciated Biosphere matrix with the scaling vector
    1. Repeat for next sector-region-combination and append results
1. Cleaning the results: Removing artifacts, applying rules to cut-off erroneous/uncertain EFs
    1. Combining calculated EFs with vector x (total output) in order to
    1. Remove sectors with either low output or very low or negative EF
    1. Calculating sectorial median; dropping EFs > upper fence (20 x sectorial median)
    1. Export final set of EFs

---

**[1]** Technically also *industry x industry* and all years 1995-2022 are possible to calculate with this script.

- *As of v3.8.2, the end years are: 2015 energy (IEA extended energy balances); 2019 all GHGs (non fuel, non-CO2 are nowcasted from 2018); 2013 material; 2011 most others, land, water. More recent data are purely "now-casts".**
- *Now-casting: The time-series extends into the future, but this is based on the now-casting procedure and projections of GDP and aggregate trade of the IMF (Intntl. Monetary Fund).*

Hence I chose that 2019 is the most up-to-date year without major distortions (e.g. no Covid-Pandemic in now-casts).

# Set up: 
## Setting year and type of raw data which will be parsed

In [5]:
exio3_year = '2019' #YYYY
exio3_type = 'pxp'  #ixi or pxp
exio3_folder = r"C:\Users\julia\Desktop\Notebook\EXIO3data" 

## Parse and calc_all()

In [2]:
import numpy as np    
import pandas as pd
pd.set_option('display.max_colwidth', None)
import pymrio
#import scipy.linalg 

In [6]:
#download data
exio_meta = pymrio.download_exiobase3(
    storage_folder=exio3_folder,
    system=exio3_type,
    years=exio3_year
)

In [7]:
expath = r"C:\Users\julia\Desktop\Notebook\EXIO3data\IOT_"+exio3_year+"_"+exio3_type+".zip"
exio3 = pymrio.parse_exiobase3(path=expath)
#exio3.meta
list(exio3.get_extensions())

['impacts', 'satellite']

In [8]:
exio3.calc_all()

# Using pre-calculated Leontief matrix L (total requirement matrix)
- `L translates to the normalized and inverted matrx A in LCA connotation`


In [37]:
df_L = exio3.L
df_L

region                                                           AT  \
sector                                                   Paddy rice   
region sector                                                         
AT     Paddy rice                                               1.0   
       Wheat                                                    0.0   
       Cereal grains nec                                        0.0   
       Vegetables, fruit, nuts                                  0.0   
       Oil seeds                                                0.0   
...                                                             ...   
WM     Membership organisation services n.e.c. (91)             0.0   
       Recreational, cultural and sporting services (92)        0.0   
       Other services (93)                                      0.0   
       Private households with employed persons (95)            0.0   
       Extra-territorial organizations and bodies               0.0   

region                                                                  \
sector                                                           Wheat   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              1.034286e+00   
       Cereal grains nec                                  3.692041e-05   
       Vegetables, fruit, nuts                            9.075155e-06   
       Oil seeds                                          7.194489e-07   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       1.285563e-06   
       Recreational, cultural and sporting services (92)  1.189858e-05   
       Other services (93)                                4.621820e-06   
       Private households with employed persons (95)      8.405823e-06   
       Extra-territorial organizations and bodies         0.000000e+00   

region                                                                      \
sector                                                   Cereal grains nec   
region sector                                                                
AT     Paddy rice                                             0.000000e+00   
       Wheat                                                  1.255974e-05   
       Cereal grains nec                                      1.014585e+00   
       Vegetables, fruit, nuts                                7.458307e-06   
       Oil seeds                                              5.554793e-07   
...                                                                    ...   
WM     Membership organisation services n.e.c. (91)           1.222302e-06   
       Recreational, cultural and sporting services (92)      1.054426e-05   
       Other services (93)                                    4.133199e-06   
       Private households with employed persons (95)          7.917990e-06   
       Extra-territorial organizations and bodies             0.000000e+00   

region                                                                            \
sector                                                   Vegetables, fruit, nuts   
region sector                                                                      
AT     Paddy rice                                                   0.000000e+00   
       Wheat                                                        1.935838e-05   
       Cereal grains nec                                            4.634264e-05   
       Vegetables, fruit, nuts                                      1.073192e+00   
       Oil seeds                                                    5.466588e-07   
...                                                                          ...   
WM     Membership organisation services n.e.c. (91)                 1.178493e-06   
       Recreational, cultural and s

# Using pre-calculated matrix M (total requirement factors of consumption)

In [ ]:
#actually yields same results as more complicated calculation, which follows
df_M = pd.DataFrame(exio3.impacts.M.loc['GHG emissions AR5 (GWP100) | GWP100 (IPCC, 2010)']/1e6)
df_M = df_M.reset_index()
df_M
df_M.to_excel(exyear+'_'+extype+'_GHG-EF_AR5_export_M.xlsx', index=False)

# Calculating GWP Emission Factors (direct + upstream GHG emissions per 1 EUR spent) for all sectors and regions
___

The GHG Emission Factors (EFs) can be calculation in two steps: 

1. by calculating a **scaling vector s** through multiplying the Leontief matrix with a demand vactor f (which is set to one for all products): **s = Lf**
2. then using a **(pseudo) Biosphere matrix B** [2] and multiplying it with the scaling vector s: **EF = Bs**


[2] the pseudo Biosphere matrix is matrix S, filtered for IPCC AR5, transposed; hence the direct GHG interventions stemming from each product.
This in combination with the total requirement matrix / Leontief matrix allows us to aggregate all supply chain emissions for each product.

## Preparation: flatten the indices for easier matrix multiplication

In [13]:
#make copy of normalized, inverted A
df_L_flat = df_L.copy()

#flatten column headers
df_L_flat.columns = df_L_flat.columns.get_level_values(0) + '_' +  df_L_flat.columns.get_level_values(1)

#flatten multiindex into separate columns
df_L_flat = df_L_flat.reset_index()

#merge new columns
df_L_flat["region_sector"] = df_L_flat["region"] + '_' + df_L_flat["sector"] 

#drop single columns
df_L_flat.drop(['region', 'sector'], axis=1, inplace=True) 

#saving for later
df_region_sector = df_L_flat["region_sector"] 

#making region_sector column new index
df_L_flat.set_index('region_sector', inplace=True) 
df_L_flat 

,AT_Paddy rice,AT_Wheat,AT_Cereal grains nec,"AT_Vegetables, fruit, nuts",AT_Oil seeds,"AT_Sugar cane, sugar beet",AT_Plant-based fibers,AT_Crops nec,AT_Cattle,AT_Pigs,...,WM_Paper for treatment: landfill,WM_Plastic waste for treatment: landfill,WM_Inert/metal/hazardous waste for treatment: landfill,WM_Textiles waste for treatment: landfill,WM_Wood waste for treatment: landfill,WM_Membership organisation services n.e.c. (91),"WM_Recreational, cultural and sporting services (92)",WM_Other services (93),WM_Private households with employed persons (95),WM_Extra-territorial organizations and bodies
region_sector,,,,,,,,,,,,,,,,,,,,,
AT_Paddy rice,1.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
AT_Wheat,0.0,1.034286e+00,1.255974e-05,1.935838e-05,8.290215e-06,0.000019,3.632077e-06,2.814186e-03,0.006529,0.004847,...,1.499540e-06,1.460563e-06,1.411737e-06,1.045025e-06,1.134792e-06,4.317146e-07,6.348631e-07,7.923880e-07,3.204404e-07,0.0
AT_Cereal grains nec,0.0,3.692041e-05,1.014585e+00,4.634264e-05,2.009453e-05,0.000047,1.059919e-05,1.942637e-03,0.040111,0.033262,...,2.341706e-06,1.864354e-06,2.279748e-06,1.450563e-06,1.542006e-06,1.087381e-06,1.621469e-06,2.464700e-06,5.248568e-07,0.0
"AT_Vegetables, fruit, nuts",0.0,9.075155e-06,7.458307e-06,1.073192e+00,4.808050e-06,0.000011,2.385988e-06,1.242110e-05,0.001563,0.001495,...,2.520373e-06,2.155282e-06,2.376953e-06,1.037357e-06,1.362083e-06,6.790456e-07,1.249391e-06,1.672362e-06,3.548328e-07,0.0
AT_Oil seeds,0.0,7.194489e-07,5.554793e-07,5.466588e-07,1.003040e+00,0.000001,2.975841e-07,9.499971e-07,0.000139,0.000127,...,7.046209e-07,7.185844e-07,7.092242e-07,6.556074e-07,6.436880e-07,2.789404e-07,3.721989e-07,4.825450e-07,1.614682e-07,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WM_Membership organisation services n.e.c. (91),0.0,1.285563e-06,1.222302e-06,1.178493e-06,7.255503e-07,0.000002,1.073504e-07,1.823688e-06,0.000003,0.000002,...,2.024517e-04,2.215154e-04,2.061557e-04,2.333251e-04,2.212841e-04,1.004832e+00,2.062423e-03,9.171476e-04,5.942938e-04,0.0
"WM_Recreational, cultural and sporting services (92)",0.0,1.189858e-05,1.054426e-05,1.035197e-05,5.936715e-06,0.000013,1.221192e-06,1.742669e-05,0.000031,0.000015,...,1.211121e-03,1.171367e-03,1.327417e-03,1.082020e-03,1.141323e-03,9.771070e-03,1.069985e+00,7.463608e-03,4.627779e-04,0.0
WM_Other services (93),0.0,4.621820e-06,4.133199e-06,3.139663e-06,2.512460e-06,0.000005,4.754927e-07,6.981925e-06,0.000012,0.000005,...,7.464956e-04,7.426962e-04,7.886586e-04,8.143954e-04,8.259900e-04,5.066943e-03,4.285283e-03,1.013673e+00,2.844315e-04,0.0


## Preparation: Creating an Identity matrix with correct index in a dataframe to use as the demand vector

In [14]:
df_I = pd.DataFrame(np.identity(df_L.shape[0]))

#introduce and set index
df_I["region_sector"] = df_region_sector 
df_I.set_index('region_sector', inplace=True)

df_I

,0,1,2,3,4,5,6,7,8,9,...,9790,9791,9792,9793,9794,9795,9796,9797,9798,9799
region_sector,,,,,,,,,,,,,,,,,,,,,
AT_Paddy rice,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AT_Wheat,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AT_Cereal grains nec,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"AT_Vegetables, fruit, nuts",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AT_Oil seeds,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WM_Membership organisation services n.e.c. (91),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
"WM_Recreational, cultural and sporting services (92)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
WM_Other services (93),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


## Preparation: Creating a depreciated, "pseudo" biosphere
- `S = Direct stressor/impact coefficients`
- divided by 1e6 to normalize to 1 EUR, therefore in single Euro; not M.EUR

In [10]:
df_S = pd.DataFrame(exio3.impacts.S.loc['GHG emissions AR5 (GWP100) | GWP100 (IPCC, 2010)']/1e6)
df_S.head(3)

GHG emissions AR5 (GWP100) | GWP100 (IPCC, 2010)
region sector                                                             
AT     Paddy rice                                                 0.000000
       Wheat                                                      1.597931
       Cereal grains nec                                          0.881842

In [11]:
#Create pseudo biosphere matrix by transposing S
pseudo_bio = df_S.T.copy()

#flatten column headers
pseudo_bio.columns = pseudo_bio.columns.get_level_values(0) + '_' +  pseudo_bio.columns.get_level_values(1)

#pseudo_bio.T.to_excel('export_pseudo_bio.xlsx')
pseudo_bio

,AT_Paddy rice,AT_Wheat,AT_Cereal grains nec,"AT_Vegetables, fruit, nuts",AT_Oil seeds,"AT_Sugar cane, sugar beet",AT_Plant-based fibers,AT_Crops nec,AT_Cattle,AT_Pigs,...,WM_Paper for treatment: landfill,WM_Plastic waste for treatment: landfill,WM_Inert/metal/hazardous waste for treatment: landfill,WM_Textiles waste for treatment: landfill,WM_Wood waste for treatment: landfill,WM_Membership organisation services n.e.c. (91),"WM_Recreational, cultural and sporting services (92)",WM_Other services (93),WM_Private households with employed persons (95),WM_Extra-territorial organizations and bodies
"GHG emissions AR5 (GWP100) | GWP100 (IPCC, 2010)",0.0,1.597931,0.881842,0.176051,1.203697,1.592273,0.312469,0.942043,5.300144,0.458335,...,22.639178,1.520093,0.173379,9.405079,2.029944,0.024813,0.058525,0.149958,0.01072,0.0


## MAIN CALCULATION: Calculate GWP EF for all sectors and regions

In [23]:
results = []

for x in range(0,df_L.shape[0]): #815,977 for DE
    #creating final/external demand vectors for each sector/product and region,
    #where only the studied sector/product is demanded, 
    #everything else is set to zero, but the supply chain for our studied sector are pulled in. 
    #it is an exogenously defined set of economic flows of which we impose
    #that the system produces exactly the given amount = 1 of our studied sector.
    demand_vector = pd.DataFrame(df_I.iloc[:, x])
    
    #step1+2
    inventory_result = pseudo_bio.dot(df_L_flat.dot(demand_vector))
    
    #results
    results.append(inventory_result.T)

In [24]:
final_results = pd.concat(results)
final_results.index = df_S.index
final_results = final_results.reset_index()
final_results.to_excel(exyear+'_'+extype+'_GHG-EF_AR5.xlsx', index=False)

# Cleaning the results: Removing artifacts
___

1. Combining calculated EFs with vector x (total output)  
2. Removing sectors with low output (indout<10e-2) or very low (<10e-5) EF, or negative EF
3. Calculating sectorial median; dropping EFs > *upper fence* (20 x sectorial median)
4. Calculating weighted sectorial averages as global fallbacks and adding them to the masterlist


## Combining calculated EFs with vector x (total output)

In [25]:
df_x = exio3.x
df_x.head(3)

indout
region sector                       
AT     Paddy rice           0.000000
       Wheat              359.797737
       Cereal grains nec  864.548099

In [26]:
df_x_flat = df_x.reset_index()
final_results_clean = final_results.copy()

#Combining calculated EFs with vector x (total output)
final_results_clean = final_results_clean.merge(df_x_flat,
                                               on=['region', 'sector'],
                                               how='left')

print(final_results_clean.count())
final_results_clean.head(3)

region                                              9800
sector                                              9800
GHG emissions AR5 (GWP100) | GWP100 (IPCC, 2010)    9800
indout                                              9800
dtype: int64


,region,sector,"GHG emissions AR5 (GWP100) | GWP100 (IPCC, 2010)",indout
0,AT,Paddy rice,0.000000,0.000000
1,AT,Wheat,1.819858,359.797737
2,AT,Cereal grains nec,1.025201,864.548099


## Removing sectors with low output or very low EF, or negative EF

In [27]:
#renaming
EF_list_drop_low_neg = final_results_clean.rename(columns={'region':'country_code',
                                    'GHG emissions AR5 (GWP100) | GWP100 (IPCC, 2010)':'factor_co2eq'
                                   })

#Removing sectors with really low output ("indout") or very low or negative EF
EF_list_drop_low_neg = EF_list_drop_low_neg.drop(EF_list_drop_low_neg[(EF_list_drop_low_neg.indout <10e-2 ) |
                                                                  (EF_list_drop_low_neg.factor_co2eq <10e-5)].index)


print(EF_list_drop_low_neg.count())
EF_list_drop_low_neg.head(3)

country_code    7844
sector          7844
factor_co2eq    7844
indout          7844
dtype: int64


,country_code,sector,factor_co2eq,indout
1,AT,Wheat,1.819858,359.797737
2,AT,Cereal grains nec,1.025201,864.548099
3,AT,"Vegetables, fruit, nuts",0.294887,1665.261805


## Calculating sectorial median; dropping EFs > *upper fence*

In [28]:
#Calculating sectorial median
medians = pd.DataFrame(EF_list_drop_low_neg.groupby(['sector']).median()['factor_co2eq']) 
medians.rename(columns={'factor_co2eq':'sector_median'}, inplace=True)
#medians = medians.reset_index()


#Dropping EFs > outer fence (20 x sectorial median)
EF_list_drop_low_neg_fenced = EF_list_drop_low_neg.merge(medians,
                                                   on='sector', how='left')


EF_list_drop_low_neg_fenced = EF_list_drop_low_neg_fenced.drop(
    EF_list_drop_low_neg_fenced[
        (EF_list_drop_low_neg_fenced.factor_co2eq > 20* EF_list_drop_low_neg_fenced.sector_median)
        ].index)

print(EF_list_drop_low_neg_fenced.count())
EF_list_drop_low_neg_fenced.head(3)

country_code     7732
sector           7732
factor_co2eq     7732
indout           7732
sector_median    7732
dtype: int64


,country_code,sector,factor_co2eq,indout,sector_median
0,AT,Wheat,1.819858,359.797737,1.656794
1,AT,Cereal grains nec,1.025201,864.548099,1.224095
2,AT,"Vegetables, fruit, nuts",0.294887,1665.261805,0.481128


## Calculating weighted sectorial averages as global fallbacks

In [29]:
totaloutput = pd.DataFrame(EF_list_drop_low_neg_fenced.groupby(['sector']).sum()['indout'])
totaloutput.rename(columns={'indout':'total_output'}, inplace=True)


EF_list_dropall_fallback = EF_list_drop_low_neg_fenced.merge(totaloutput,
                                                   on='sector', how='left')
print(EF_list_dropall_fallback.count())
EF_list_dropall_fallback.head(3)

country_code     7732
sector           7732
factor_co2eq     7732
indout           7732
sector_median    7732
total_output     7732
dtype: int64


,country_code,sector,factor_co2eq,indout,sector_median,total_output
0,AT,Wheat,1.819858,359.797737,1.656794,2.312682e+05
1,AT,Cereal grains nec,1.025201,864.548099,1.224095,3.593261e+05
2,AT,"Vegetables, fruit, nuts",0.294887,1665.261805,0.481128,1.109827e+06


In [30]:
partial_weight = []
for index, row in EF_list_dropall_fallback.iterrows():
    partial_weight = EF_list_dropall_fallback['indout'] / EF_list_dropall_fallback['total_output']

EF_list_dropall_fallback["weight"] = partial_weight

weighted_EF = []
for index, row in EF_list_dropall_fallback.iterrows():
    weighted_EF = EF_list_dropall_fallback['factor_co2eq'] * EF_list_dropall_fallback['weight']

EF_list_dropall_fallback["weighted_EF"] = weighted_EF

print(EF_list_dropall_fallback.count())
EF_list_dropall_fallback.head(3)

country_code     7732
sector           7732
factor_co2eq     7732
indout           7732
sector_median    7732
total_output     7732
weight           7732
weighted_EF      7732
dtype: int64


,country_code,sector,factor_co2eq,indout,sector_median,total_output,weight,weighted_EF
0,AT,Wheat,1.819858,359.797737,1.656794,2.312682e+05,0.001556,0.002831
1,AT,Cereal grains nec,1.025201,864.548099,1.224095,3.593261e+05,0.002406,0.002467
2,AT,"Vegetables, fruit, nuts",0.294887,1665.261805,0.481128,1.109827e+06,0.001500,0.000442


In [33]:
#summing up the sectorial averages
glo_wei_avg = pd.DataFrame(EF_list_dropall_fallback.groupby(['sector']).sum()['weighted_EF'])
glo_wei_avg = glo_wei_avg.reset_index()
glo_wei_avg.rename(columns={'weighted_EF':'factor_co2eq'}, inplace=True)
glo_wei_avg["country_code"] = 'Null'
glo_wei_avg 
cols_to_order = ['country_code', 'sector', 'factor_co2eq']
glo_wei_avg = glo_wei_avg[cols_to_order]

print(glo_wei_avg.count())
glo_wei_avg.head(3) 

country_code    184
sector          184
factor_co2eq    184
dtype: int64


,country_code,sector,factor_co2eq
0,Null,Additives/Blending Components,1.229987
1,Null,Air transport services (62),1.306360
2,Null,Aluminium and aluminium products,1.516780


In [34]:
#dropping unnecessary columns 
EF_list_dropall_fallback.drop(columns=['indout', 'sector_median', 'total_output', 'weight', 'weighted_EF'], inplace=True)
EF_list_dropall_fallback = pd.concat([EF_list_dropall_fallback, glo_wei_avg])

print(EF_list_dropall_fallback.count())
EF_list_dropall_fallback.head(3)

country_code    7916
sector          7916
factor_co2eq    7916
dtype: int64


,country_code,sector,factor_co2eq
0,AT,Wheat,1.819858
1,AT,Cereal grains nec,1.025201
2,AT,"Vegetables, fruit, nuts",0.294887


## Export

In [35]:
EF_list_dropall_fallback.to_excel('2019_pxp_GHG-EF_AR5_clean.xlsx', index=False)

# Annex

**Info**
- **As of v3.8.2, the end years are: 2015 energy (IEA extended energy balances); 2019 all GHGs (non fuel, non-CO2 are nowcasted from 2018); 2013 material; 2011 most others, land, water. More recent data are purely "now-casts".**
- ***Now-casting*: The time-series extends into the future, but this is based on the now-casting procedure and projections of GDP and aggregate trade of the IMF (Intntl. Monetary Fund).**
___
- The International Energy Agency’s extended **energy balances** are the starting point to generate energy accounts, which have to be transformed (alignment with the *residence principle of SEEA* (System of Economic-Environmental Accounting)). Energy flows and products are then allocated to EXIOBASE products by means of a variety of auxiliary datasets. (The resulting physical energy flow accounts contain separate matrices depicting the supply and use of natural inputs, products and energy residuals in accordance with the overall scheme described in the SEEA.)
___
- **Emissions to air** are one of the main extensions in EXIOBASE. These have been calculated at the global level in a consistent way for all countries and sectors to the extent possible, covering each country individually and the full time series. (Used datasets: datasets which have complete or partial sets of global emissions to air on a territorial basis (e.g. IIASA GAINS, JRC EDGAR, as well as official reported air emissions by individual countries to International Conventions), and also the environmental accounts reported to Eurostat.)
- The calculation of air emissions has been conducted by combining activity data with consolidated emission factors retrieved from the TEAM model (Pulles et al. 2007). This model has been filled with emission factors from various sources, including from the Guidelines for national Greenhouse Gas and Air Pollutant Inventories (IPCC 2015; European Environment Agency 2009) and from the GAINS model (Amann 2009; IIASA 2013). The model chooses for each activity the most appropriate technology or set of technologies. The main advantage of using this model is that it allows for the introduction of new (mostly cleaner) technologies over time, thus changing the emission factors associated with certain activities.
- For emissions arising from combustion processes energy use data is combined with emission factors obtained from the TEAM model. In order to do so, the energy balances from the International Energy Agency (IEA 2013a; IEA 2013b) have to be aligned with the system boundaries described in the SEEA. The resulting emissions are then allocated to the EXIOBASE 3 industries, final consumption categories and product groups based on auxiliary datasets.
- For emissions resulting from non-combustion activities, activity data are collected from various sources (e.g. UN Statistics, USGS, BGS, FAOSTAT, etc.) and combined with one or more chosen technologies, similar to the combustion emissions. The non-combustion emissions are then each associated to one or more DESIRE products, either in the supply or the use of these products.
___
- For the compilation of the **material** extensions in EXIOBASE 3, the main basic data source for the environmental extensions related to material extraction was the SERI/WU Global Material Flow Database (available at www.materialflows.net, SERI and WU, 2014). This database currently covers more than 300 different types of biotic and abiotic raw materials and more than 200 countries.